In [211]:
import datetime, random

MAX_TIMESTAMP = 2**32 - 1
MIN_TIMESTAMP = -(2**32)
n = 10000

X = [datetime.date.fromtimestamp(random.randint(MIN_TIMESTAMP, MAX_TIMESTAMP)) for _ in range(n)]
Y = [date.strftime('%A %B %d, %Y') for date in X]

X = [str(x) for x in X]

max_len_x = max([len(x) for x in X])
max_len_y = max([len(y) for y in Y])
max_len = max(max_len_x, max_len_y)

X = ['<' + x + ''.join(['>' for _ in range(max_len - len(x) + 1)]) for x in X]
Y = ['<' + y + ''.join(['>' for _ in range(max_len - len(y) + 1)]) for y in Y]

print(X[0], Y[0])

<1905-04-18>>>>>>>>>>>>>>>>>>> <Tuesday April 18, 1905>>>>>>>


In [212]:
import tensorflow as tf

tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([x for x in X] + Y)
num_tokens = len(tokenizer.word_index)
print(num_tokens)

37


In [213]:
def dsprint(ds):
  for item in ds.take(1):
    print(item)

def preprocess(date, max_len):
  return tokenizer.texts_to_sequences(date)

X_p = [preprocess(x, max_len) for x in X]
Y_p = [preprocess(y, max_len) for y in Y]

ds = tf.data.Dataset.from_generator(lambda: [(X_p[i], Y_p[i]) for i in range(n)], output_signature=(
  tf.TensorSpec(shape=(max_len + 2, 1), dtype=tf.int32),
  tf.TensorSpec(shape=(max_len + 2, 1), dtype=tf.int32),
))
ds = ds.map(lambda x, y: ((x, x), y))
ds = ds.batch(32).prefetch(1)

dsprint(ds)

((<tf.Tensor: shape=(32, 30, 1), dtype=int32, numpy=
array([[[ 6],
        [ 3],
        [ 9],
        [ 4],
        [22],
        [ 7],
        [ 4],
        [20],
        [ 7],
        [ 3],
        [13],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1]],

       [[ 6],
        [ 3],
        [ 9],
        [ 9],
        [ 4],
        [ 7],
        [ 4],
        [ 9],
        [ 7],
        [ 4],
        [13],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1]],

       [[ 6],
        [ 3],
        [ 9],
        [ 5],
        [19],
        [ 7],
        [ 4],
     

In [214]:
# Example of encoder-decoder architecture
import tensorflow_addons as tfa

embed_size = 20
vocab_size = num_tokens

encoder_inputs = tf.keras.layers.Input(shape=[None], dtype=tf.int32)
decoder_inputs = tf.keras.layers.Input(shape=[None], dtype=tf.int32)
# sequence_lengths = tf.keras.layers.Input(shape=[], dtype=tf.int32)

embeddings = tf.keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = tf.keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.TrainingSampler()

decoder_cell = tf.keras.layers.LSTMCell(512)
output_layer = tf.keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler, output_layer=output_layer)

final_outputs, final_state, _ = decoder(decoder_embeddings, initial_state=encoder_state, sequence_length=(max_len, max_len))
y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=[y_proba])

model.compile(optimizer='adam', loss=[tf.keras.losses.sparse_categorical_crossentropy], metrics=[tf.keras.metrics.sparse_categorical_accuracy])


In [215]:
model.fit(ds, epochs=2)

Epoch 1/2


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "/opt/homebrew/Cellar/python@3.10/3.10.9/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/homebrew/Cellar/python@3.10/3.10.9/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Users/carlschader/programming/ml-tensorflow/.venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/carlschader/programming/ml-tensorflow/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1041, in launch_instance
      app.start()
    File "/Users/carlschader/programming/ml-tensorflow/.venv/lib/python3.10/site-packa